In [1]:
import cv2
import mediapipe as mp
import os

2023-05-30 20:56:43.184988: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-30 20:56:43.373952: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/guilherme/.local/lib/python3.10/site-packages/cv2/../../lib64:
2023-05-30 20:56:43.373978: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-05-30 20:56:44.418239: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dl

In [2]:
def get_bounding_rect(results, image_shape):
    for hand_landmarks in results.multi_hand_landmarks:
        x_max = 0
        x_min = image_shape[1]
        y_max = 0
        y_min = image_shape[0]

        for landmark in hand_landmarks.landmark:
            x = min(image_shape[1], max(0, int(landmark.x * image_shape[1])))
            y = min(image_shape[0], max(0, int(landmark.y * image_shape[0])))

            x_max = max(x_max, x)
            x_min = min(x_min, x)
            y_max = max(y_max, y)
            y_min = min(y_min, y)

        return x_min, y_min, x_max - x_min, y_max - y_min


In [72]:
def capture_hand_image():
    folder_path = os.path.abspath('./Data/Fold1/Y')
    
    mp_drawing = mp.solutions.drawing_utils
    mp_hands = mp.solutions.hands
    counter = 0

    # Inicializa a câmera
    cap = cv2.VideoCapture(0)

    # Configuração da mediapipe
    with mp_hands.Hands(
        static_image_mode=True,
        max_num_hands=2,
        min_detection_confidence=0.7
    ) as hands:
        while cap.isOpened():
            ret, frame = cap.read()

            # Convertendo o frame para RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            # Detecta as mãos no frame
            results = hands.process(image)

            # Desenha as landmarks da mão no frame
            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:
                    mp_drawing.draw_landmarks(
                        image,
                        hand_landmarks,
                        mp_hands.HAND_CONNECTIONS
                    )

            # Exibe o frame
            cv2.imshow('Hand Detection', cv2.cvtColor(image, cv2.COLOR_RGB2BGR))

            # Captura a imagem quando a tecla 's' for pressionada
            if cv2.waitKey(10) & 0xFF == ord('s'):
                # Encontra os limites do retângulo que envolve a mão
                x, y, w, h = get_bounding_rect(results, frame.shape[:2])

                # Recorta a região da mão na imagem original
                hand_image = frame[y-30:y+h+50, x-30:x+w+30]

                # Salva a imagem da mão isolada
                cv2.imwrite(f"{folder_path}/{counter}.jpg", cv2.cvtColor(hand_image, cv2.COLOR_RGB2BGR))
                counter += 1

            if(counter > 9) : break
        # Libera os recursos
        cap.release()
        cv2.destroyAllWindows()


In [73]:
if __name__ == "__main__":
    capture_hand_image()